## `text_classification.ipynb` - классификация по жанрам фильмов

**Блокнот анализирует жанры фильмов, упрощая их классификацию, с помощью zero-shot classification модели предсказывает жанры для пропущенных значений**

In [1]:
import pandas as pd

movies = pd.read_csv("../data/movies_cleaned.csv")
movies.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,tagged_overview
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...","tt1375666 Cobb, a skilled thief who commits co..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",tt0816692 The adventures of a group of explore...
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",tt0468569 Batman raises the stakes in his war ...
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...","tt0499549 In the 22nd century, a paraplegic Ma..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",tt0848228 When an unexpected enemy emerges and...


**Подсчитываем, сколько раз каждый жанр встречается в датасете**

In [2]:
movies["genres"].value_counts().reset_index()

,genres,count
0,Drama,38
1,"Action, Adventure, Science Fiction",37
2,Comedy,33
3,"Drama, Romance",22
4,"Comedy, Romance",22
...,...,...
538,"Drama, Action, Fantasy",1
539,"Drama, Horror, Thriller",1
540,"Action, Adventure, Horror, Science Fiction",1
541,"Adventure, Horror, Comedy",1


**Фильтруем жанры по частоте фильмов**

In [3]:
movies["genres"].value_counts().reset_index().query("count > 5")

,genres,count
0,Drama,38
1,"Action, Adventure, Science Fiction",37
2,Comedy,33
3,"Drama, Romance",22
4,"Comedy, Romance",22
5,"Action, Adventure, Fantasy",21
6,"Adventure, Action, Science Fiction",20
7,"Action, Thriller",14
8,"Action, Thriller, Science Fiction",13
9,"Drama, Crime",13


**Подсчитываем жанры по отдельности**

In [4]:
all_genres = movies["genres"].str.split(', ').explode()

genre_counts = all_genres.value_counts().reset_index()
genre_counts.columns = ["genre", "count"]
genre_counts

,genre,count
0,Action,491
1,Adventure,421
2,Drama,410
3,Comedy,340
4,Thriller,327
5,Science Fiction,280
6,Fantasy,251
7,Family,197
8,Crime,178
9,Animation,151


**Преобразуем жанры в более упрощенную, сбалансированную категорию**

In [5]:
def balanced_simple_genre(genre_string):
    if pd.isna(genre_string):
        return "Other"

    genres = genre_string.split(", ")

    if "Action" in genres or "Adventure" in genres:
        return "Action/Adventure"
    elif "Science Fiction" in genres:
        return "Sci-Fi"
    elif "Fantasy" in genres:
        return "Fantasy"
    elif "Comedy" in genres:
        return "Comedy"
    elif "Drama" in genres:
        return "Drama"
    elif "Horror" in genres:
        return "Horror"
    elif "Thriller" in genres:
        return "Thriller"
    elif "Romance" in genres:
        return "Romance"
    elif "Animation" in genres:
        return "Animation"
    elif "Documentary" in genres:
        return "Documentary"
    elif "Crime" in genres or "Mystery" in genres:
        return "Crime/Mystery"
    elif "Family" in genres:
        return "Family"
    else:
        return "Other"


**Применяем функцию ко всему датасету**

In [6]:
movies["simple_genres"] = movies["genres"].apply(balanced_simple_genre)
movies

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,tagged_overview,simple_genres
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...","tt1375666 Cobb, a skilled thief who commits co...",Action/Adventure
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",tt0816692 The adventures of a group of explore...,Action/Adventure
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",tt0468569 Batman raises the stakes in his war ...,Action/Adventure
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...","tt0499549 In the 22nd century, a paraplegic Ma...",Action/Adventure
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",tt0848228 When an unexpected enemy emerges and...,Action/Adventure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,529203,The Croods: A New Age,7.521,3508,Released,2020-11-25,215905815,95,False,/ytTQoYkdpsgtfDWrNFCei8Mfbxu.jpg,...,63.456,/tbVZ3Sq88dZaCANlUcewQuHQOaE.jpg,The future ain't what it used to be.,"Animation, Family, Adventure, Fantasy, Comedy","Universal Pictures, DreamWorks Animation",United States of America,English,"sequel, prehistory, candid, playful, joyous, a...","tt2850386 Searching for a safer habitat, the p...",Action/Adventure
1224,9392,The Descent,6.957,3507,Released,2005-07-08,57130027,99,False,/70TIOrfkQli0Smsfjua2McaDPci.jpg,...,23.007,/mxFPI4KYBk5ri9cPteIS8jiDFgj.jpg,Scream your last breath.,"Adventure, Horror","Pathé, Celador Films, Northmen Productions",United Kingdom,English,"panic, darkness, mutant, expedition, cave, cla...","tt0435625 After a tragic accident, six friends...",Action/Adventure
1225,12153,White Chicks,6.919,3505,Released,2004-06-23,113086475,109,False,/di47xqYMCYpjqwnqNlO17X5qXMX.jpg,...,54.851,/aHTUpo45qy9QYIOnVITGGqLoVcA.jpg,They're going deep undercover.,"Comedy, Crime","Columbia Pictures, Revolution Studios, Wayans ...",United States of America,English,"undercover, fbi, cross dressing, car accident,...","tt0381707 Two FBI agent brothers, Marcus and K...",Comedy
1226,2832,Identity,7.180,3502,Released,2003-04-25,90259536,90,False,/7MwDOMrbjrKP3XQ5vw4cgB2DPaF.jpg,...,27.747,/bnidwEvWNAVJ3Uco9wWtuzWAfrx.jpg,The secret lies within.,"Mystery, Thriller","Konrad Pictures, Columbia Pictures

In [7]:
movies[~(movies["simple_genres"].isna())]

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,tagged_overview,simple_genres
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...","tt1375666 Cobb, a skilled thief who commits co...",Action/Adventure
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",tt0816692 The adventures of a group of explore...,Action/Adventure
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",tt0468569 Batman raises the stakes in his war ...,Action/Adventure
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...","tt0499549 In the 22nd century, a paraplegic Ma...",Action/Adventure
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",tt0848228 When an unexpected enemy emerges and...,Action/Adventure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,529203,The Croods: A New Age,7.521,3508,Released,2020-11-25,215905815,95,False,/ytTQoYkdpsgtfDWrNFCei8Mfbxu.jpg,...,63.456,/tbVZ3Sq88dZaCANlUcewQuHQOaE.jpg,The future ain't what it used to be.,"Animation, Family, Adventure, Fantasy, Comedy","Universal Pictures, DreamWorks Animation",United States of America,English,"sequel, prehistory, candid, playful, joyous, a...","tt2850386 Searching for a safer habitat, the p...",Action/Adventure
1224,9392,The Descent,6.957,3507,Released,2005-07-08,57130027,99,False,/70TIOrfkQli0Smsfjua2McaDPci.jpg,...,23.007,/mxFPI4KYBk5ri9cPteIS8jiDFgj.jpg,Scream your last breath.,"Adventure, Horror","Pathé, Celador Films, Northmen Productions",United Kingdom,English,"panic, darkness, mutant, expedition, cave, cla...","tt0435625 After a tragic accident, six friends...",Action/Adventure
1225,12153,White Chicks,6.919,3505,Released,2004-06-23,113086475,109,False,/di47xqYMCYpjqwnqNlO17X5qXMX.jpg,...,54.851,/aHTUpo45qy9QYIOnVITGGqLoVcA.jpg,They're going deep undercover.,"Comedy, Crime","Columbia Pictures, Revolution Studios, Wayans ...",United States of America,English,"undercover, fbi, cross dressing, car accident,...","tt0381707 Two FBI agent brothers, Marcus and K...",Comedy
1226,2832,Identity,7.180,3502,Released,2003-04-25,90259536,90,False,/7MwDOMrbjrKP3XQ5vw4cgB2DPaF.jpg,...,27.747,/bnidwEvWNAVJ3Uco9wWtuzWAfrx.jpg,The secret lies within.,"Mystery, Thriller","Konrad Pictures, Columbia Pictures

**Подготавливаем инструменты для автоматической классификации фильмов по жанру с помощью предобученной модели `facebook/bart-large-mnli`**

In [8]:
from transformers import pipeline
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_categories = ["Action Movie", "Not Action Movie"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

Device set to use cuda


**Извлекаем первый фильм с жанром _Action/Adventure_**

In [9]:
sequence = movies.loc[movies["simple_genres"] == "Action/Adventure", "overview"].reset_index(drop=True)[0]

**Запускаем zero-shot classification для данного фильма**

In [10]:
pipe(sequence, binary_categories)

{'sequence': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.',
 'labels': ['Not Action Movie', 'Action Movie'],
 'scores': [0.7621052861213684, 0.2378946989774704]}

**На основе выхода классификатора определяем наиболее вероятную категорию**

In [11]:
import numpy as np

max_index = np.argmax(pipe(sequence, binary_categories)["scores"])
max_label = pipe(sequence, binary_categories)["labels"][max_index]
max_label

'Not Action Movie'

**На основании функции `generate_predictions` автоматизируем получение предсказания категории текста**

In [12]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

**Формируем наборы данных для оценки качества классификации фильмов с помощью языковой модели**

In [13]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 100)):
    sequence = movies.loc[movies["simple_genres"] == "Action/Adventure", "overview"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, binary_categories)]
    actual_cats += ["Action Movie"]

100%|██████████| 100/100 [00:06<00:00, 15.09it/s]


In [14]:
for i in tqdm(range(0, 50)):
    sequence = movies.loc[movies["simple_genres"] == "Drama", "overview"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, binary_categories)]
    actual_cats += ["Not Action Movie"]

100%|██████████| 50/50 [00:03<00:00, 16.34it/s]


In [15]:
for i in tqdm(range(0, 5)):
    sequence = movies.loc[movies["simple_genres"] == "Thriller", "overview"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, binary_categories)]
    actual_cats += ["Not Action Movie"]

100%|██████████| 5/5 [00:00<00:00, 15.56it/s]


In [16]:
for i in tqdm(range(0, 5)):
    sequence = movies.loc[movies["simple_genres"] == "Horror", "overview"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, binary_categories)]
    actual_cats += ["Not Action Movie"]

100%|██████████| 5/5 [00:00<00:00, 16.30it/s]


**Создаем датафрейм с результатами классификации для дальнешего анализа**

In [17]:
predictions_df = pd.DataFrame({"actual": actual_cats, "predicted": predicted_cats})
predictions_df

,actual,predicted
0,Action Movie,Not Action Movie
1,Action Movie,Action Movie
2,Action Movie,Action Movie
3,Action Movie,Not Action Movie
4,Action Movie,Action Movie
...,...,...
155,Not Action Movie,Not Action Movie
156,Not Action Movie,Not Action Movie
157,Not Action Movie,Not Action Movie
158,Not Action Movie,Not Action Movie


**Добавляем столбец `corrected_prediction`, который показывает совпадение/несовпадение предсказания модели с истинной меткой**

In [18]:
predictions_df["corrected_prediction"] = np.where(predictions_df["actual"] == predictions_df["predicted"], 1, 0)
predictions_df

,actual,predicted,corrected_prediction
0,Action Movie,Not Action Movie,0
1,Action Movie,Action Movie,1
2,Action Movie,Action Movie,1
3,Action Movie,Not Action Movie,0
4,Action Movie,Action Movie,1
...,...,...,...
155,Not Action Movie,Not Action Movie,1
156,Not Action Movie,Not Action Movie,1
157,Not Action Movie,Not Action Movie,1
158,Not Action Movie,Not Action Movie,1


**Находим точность модели**

In [19]:
predictions_df["corrected_prediction"].sum() / len(predictions_df)

np.float64(0.54375)

**Подготавливаем данные для предсказания жанров фильмов, у которых отсутствует разметка**

In [20]:
imdb_ids = []
predicted_cats = []

missing_cats = movies.loc[movies["simple_genres"].isna(), ["imdb_id", "overview"]].reset_index(drop=True)

**Делаем предсказание**

In [21]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["overview"][i]
    predicted_cats += [generate_predictions(sequence, binary_categories)]
    imdb_ids += [missing_cats["imdb_id"][i]]

0it [00:00, ?it/s]


**Объединяем предсказанные жанры с идентификаторами**

In [22]:
missing_predicted_df = pd.DataFrame({"imdb_id": imdb_ids, "predicted_category": predicted_cats})

In [23]:
missing_predicted_df

,imdb_id,predicted_category


**Обновляем исходный датафрейм**

In [24]:
movies = pd.merge(movies, missing_predicted_df, on="imdb_id", how="left")
movies["simple_genres"] = np.where(movies["simple_genres"].isna(), movies["predicted_category"], movies["simple_genres"])
movies = movies.drop(columns = ["predicted_category"])

**Фильтруем `movies`**

In [25]:
movies[movies["genres"].str.lower().isin([
    "action",
    "sci-fi",
    "fantasy",
    "horror",
    "thriller",
    "comedy",
    "drama"
])]

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,tagged_overview,simple_genres
7,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o...",tt0137523 A ticking-time-bomb insomniac and a ...,Drama
93,18785,The Hangover,7.312,15812,Released,2009-06-02,469310836,100,False,/2o0PKGmnSgCGkzoSePNAqse8Ure.jpg,...,72.655,/uluhlXubGu1VxU63X9VHCLWDAYP.jpg,Some guys just can't handle Vegas.,Comedy,"Legendary Pictures, Green Hat Films, Warner Br...","Germany, United States of America",English,"blackjack, stag night, lost weekend, chapel, h...",tt1119646 When three friends finally come to a...,Comedy
210,14,American Beauty,8.000,11260,Released,1999-09-15,356296601,122,False,/DztBnZaqmla2sGUW9s8AyOmskT.jpg,...,29.837,/wby9315QzVKdW9BonAefg8jGTTb.jpg,Look closer.,Drama,"Jinks/Cohen Company, DreamWorks Pictures",United States of America,English,"estate agent, adultery, coming out, first time...","tt0169547 Lester Burnham, a depressed suburban...",Drama
212,37799,The Social Network,7.358,11190,Released,2010-10-01,224920315,121,False,/2BNKxbq4muNcwTjSDNCYnvr1dM8.jpg,...,36.101,/n0ybibhJtQ5icDqTp8eRytcIHJx.jpg,You don't get to 500 million friends without m...,Drama,"Columbia Pictures, Relativity Media, Scott Rud...",United States of America,English,"hacker, based on novel or book, boston, massac...",tt1285016 The tale of a new breed of cultural ...,Drama
215,489,Good Will Hunting,8.149,11111,Released,1997-12-05,225933435,127,False,/bpV8wn48s82au37QyUJ51S7X2Vf.jpg,...,56.169,/bABCBKYBK7A5G1x0FzoeoNfuj2.jpg,"Some people can never believe in themselves, u...",Drama,"Miramax, Lawrence Bender Productions, Be Gentl...",United States of America,English,"boston, massachusetts, professor, baseball, ma...",tt0119217 Will Hunting has a genius-level IQ b...,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,571,The Birds,7.519,3685,Released,1963-03-28,11500000,119,False,/q2XxGMCO8Zey8YDrvQGi8P8ns5F.jpg,...,22.267,/z0iYrJ6GsAMP3abOha7uGMuc5kZ.jpg,"...and remember, the next scream you hear may ...",Horror,Alfred J. Hitchcock Productions,United States of America,English,"pet shop, seclusion, playground, fireplace, se...",tt0056869 Chic socialite Melanie Daniels enjoy...,Horror
1177,406,La Haine,8.087,3612,Released,1995-05-31,15300000,98,False,/viGCwLxupyoaLsa6hrnu1gYrB0B.jpg,...,15.397,/gfdhSzPyAWtAizqs4ytc0MwOlQg.jpg,"How far you fall doesn't matter, it's how you ...",Drama,"Kasso Productions, La Sept Cinéma, Les Product...",France,French,"neo-nazism, police brutality, hip-hop, paris, ...",tt0113247 After a chaotic night of rioting in ...,Drama
1182,6145,Fracture,7.277,3599,Released,2007-04-19,91354215,113,False,/mujUrk2diGe5vRCb3kdpHZeobRs.jpg,...,23.784,/qNen8x5gaikjIg9CFihgxYcJwQe.jpg,I shot my wife... prove it.,Thriller,"M7 Filmproduktion, New Line Cinema, Castle Roc...","Germany, United States of America",English,"perfect crime, prosecution, legal thriller",tt0488120 A husband is on trial for the attemp...,Thriller
1191,764,The Evil Dead,7.307,3570,Released,1981-09-10,29612367,85,False,/2OgSIyerY8zUNw8czWiIn6D4kos.jpg,...,39.224,/uYxQ6xhP3WjqPZtxyAOnZQWnZqn.jpg,The Ultimate Experience In Grueling Terror,Horror,Renaissance Pictures,United States of America,English,"falsely accused, audio tape, aggression, log c...",tt0083907 Five vacationing college students un...,Horror


**Сохраняем по указанному пути**

In [26]:
movies.to_csv("../data/movies_with_genres.csv", index=False)